In [1]:
import pandas as pd
import numpy as np
from numba import njit
from time import time

In [2]:
df = pd.read_csv('data.csv', delimiter=',')
df

,id,region,municipality,settlement,type,latitude_dd,longitude_dd
0,0,Республика Алтай,Шебалинский район,Каспа,с,5111,8601
1,1,Алтайский край,Смоленский,Молочный,п,5241,8497
2,2,Красноярский край,Казачинский район,Отношка,с,5738,9270
3,3,Республика Тыва,Каа-Хемский кожуун,Кундустуг,с,5157,9518
4,4,Красноярский край,Курагинский район,Щетинкино,с,5453,9344
...,...,...,...,...,...,...,...
6132,6132,Новосибирская область,Сузунский район,Мереть,с,5357,8239
6133,6133,Новосибирская область,Сузунский район,Маюрово,с,5431,8242
6134,6134,Алтайский край,Быстроистокский,Смоленский,п,5221,8466
6135,6135,Новосибирская область,Искитимский район,Малиновка,д,5466,8386


In [8]:
@njit(fastmath=True)
def count_matrix(n, array):
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            matrix[i, j] = np.sqrt(np.sum(np.square(array[i] - array[j])))
    return matrix

@njit(fastmath=True)
def find_way(matrix, start_vertex):
    n = matrix.shape[0]
    visited = np.zeros(n, dtype=np.int64)
    edges = np.zeros(n + 1, dtype=np.int64)
    edges[n] = start_vertex
    min_distance = np.inf

    for i in range(n):
        visited[start_vertex] = 1
        min_vertex = -1
        for j in range(n):
            if j != start_vertex and not visited[j]:
                distance = matrix[start_vertex, j]
                if distance < min_distance:
                    min_distance = distance
                    min_vertex = j

        edges[i] = start_vertex
        start_vertex = min_vertex
        min_distance = np.inf

    return edges.astype("int")

def save_csv(matrix, way):
    dists = matrix[way[:-1], way[1:]] / 100
    ids = way[:-1]
    df = pd.DataFrame({'Id': ids, 'Predicted': dists.ravel()})
    df = df.astype({'Id': int, 'Predicted': float}).reset_index(drop=True)
    display(df)
    df.to_csv("res.csv", index=False, sep=",", float_format="%.3f")

In [ ]:
start = time()

arr = np.column_stack((df['latitude_dd'].to_numpy(), df['longitude_dd'].to_numpy()))
matrix = count_matrix(len(arr), arr)
way = find_way(matrix, 3753)

length = sum([matrix[way[i]][way[i + 1]] for i in range(0, len(way) - 1)]) / 100

end = time()

print(f"Длина пути: {length:.3f}")
print(f"Поиск выполнен за {end - start:.3f} seconds")

In [9]:

save_csv(matrix, way)

,Id,Predicted
0,3753,0.050990
1,2533,0.092195
2,3483,0.050990
3,622,0.020000
4,6044,0.058310
...,...,...
6132,986,13.021743
6133,3336,7.095583
6134,5629,2.457906
6135,4674,2.917070
